<a href="https://colab.research.google.com/github/Ijattttt/Juliuta/blob/main/Lab_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, pandas, absl.logging, warnings, tempfile, urllib, zipfile

import tensorflow as tf

absl.logging.set_verbosity(absl.logging.ERROR)

warnings.filterwarnings('ignore')

tf.get_logger().setLevel('ERROR')

In [ ]:
dataurl = 'data/diabetic_data.csv'

data = pandas.read_table(dataurl, sep=',')

data.isnull().sum().sum()

0

In [ ]:
data.iloc[:5, :8]

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25
1,149190,55629189,Caucasian,Female,[10-20),?,1,1
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1
3,500364,82442376,Caucasian,Male,[30-40),?,1,1
4,16680,42519267,Caucasian,Male,[40-50),?,1,1


In [ ]:
data.shape

(101766, 50)

In [ ]:
train_len = int(len(data) * 0.7)

test_len = (len(data) - train_len) // 2

train_len, test_len

(71236, 15265)

In [ ]:
train = data.iloc[:train_len].reset_index(drop = True)

test = data.iloc[train_len:train_len + test_len].reset_index(drop = True)

evaluate = data.iloc[train_len + test_len :].drop('readmitted', axis='columns').reset_index(drop = True)

train.shape[0], test.shape[0], evaluate.shape[0]

(71236, 15265, 15265)

In [ ]:
import tensorflow_data_validation as tfdv

deletes = ['encounter_id', 'patient_nbr']

usecols = [col for col in data.columns if col not in deletes]

In [ ]:
stats_options = tfdv.StatsOptions(feature_allowlist=usecols)

train_stats = tfdv.generate_statistics_from_dataframe(train, stats_options=stats_options)

len(train_stats.datasets[0].features)

48

In [ ]:
# tfdv.visualize_statistics(train_stats)

In [ ]:
test_stats = tfdv.generate_statistics_from_dataframe(test, stats_options=stats_options)

evaluate_stats = tfdv.generate_statistics_from_dataframe(evaluate, stats_options=stats_options)

In [ ]:
schemas = tfdv.infer_schema(train_stats)

# tfdv.display_schema(schemas)

In [ ]:
assert len(schemas.feature) == len(train_stats.datasets[0].features), 'Fail !'

In [ ]:
statlist = {}

statlist['lhs_statistics'] = test_stats; statlist['lhs_name'] = 'Test'

statlist['rhs_statistics'] = train_stats; statlist['rhs_name'] = 'Train'

# tfdv.visualize_statistics(**statlist)

In [ ]:
train["glimepiride-pioglitazone"].nunique()

1

In [ ]:
test["glimepiride-pioglitazone"].nunique()

2

In [ ]:
evaluate["glimepiride-pioglitazone"].nunique()

1

In [ ]:
from tensorflow_data_validation import validate_statistics, display_anomalies

def AnomaliesValidation(stats, schemas):
  """
  TensorFlow Data Validation Anomalies Validation
  """
  anomalies = validate_statistics(stats, schemas)

  total = len(anomalies.anomaly_info.items())

  if total == 0:

    return total

  display_anomalies(anomalies)

In [ ]:
AnomaliesValidation(test_stats, schemas)

`Glimepiride Pioglitazone`

In [ ]:
glimepioglita_domain = tfdv.get_domain(schemas, 'glimepiride-pioglitazone')

glimepioglita_domain.value

['No']

In [ ]:
glimepioglita_domain.value.append('Steady')

glimepioglita_domain.value

['No', 'Steady']

`Medical Specialty`

In [ ]:
medispeciality_domain = tfdv.get_domain(schemas, 'medical_specialty')

assert 'Neurophysiology' not in medispeciality_domain.value, 'False !'

In [ ]:
medispeciality_domain.value.append('Neurophysiology')

assert 'Neurophysiology' in medispeciality_domain.value, 'False !'

In [ ]:
AnomaliesValidation(test_stats, schemas)

0

In [ ]:
AnomaliesValidation(evaluate_stats, schemas)

In [ ]:
payer_code_feats = tfdv.get_feature(schemas, 'payer_code')

payer_code_feats.distribution_constraints.min_domain_mass = 0.9

In [ ]:
medical_specialty_feats = tfdv.get_feature(schemas, 'medical_specialty')

medical_specialty_feats.distribution_constraints.min_domain_mass = 0.9

In [ ]:
AnomaliesValidation(evaluate_stats, schemas)

`Repaglinide`

In [ ]:
repaloc = data.columns.get_loc('repaglinide')

ln = 22

cols_feats = data.columns.tolist()[repaloc : repaloc + ln]

In [ ]:
import logging

logging.getLogger().setLevel(logging.ERROR)

for col in cols_feats:

    tfdv.set_domain(schemas, col, 'metformin')

In [ ]:
AnomaliesValidation(evaluate_stats, schemas)

In [ ]:
schemas.default_environment.append('TRAINING')

schemas.default_environment.append('SERVING')

In [ ]:
tfdv.get_feature(schemas, 'readmitted').not_in_environment.append('SERVING')

In [ ]:
anomalies_serving = tfdv.validate_statistics(evaluate_stats, schemas, environment='SERVING')

len(anomalies_serving.anomaly_info.items())

0